<a href="https://colab.research.google.com/github/Chandrashekar0123/CGPA-Calculator/blob/main/CGPA_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('hello world')

hello world


# **Problem Statement:**
Develop a CGPA Calculator & Performance Prediction Web Application using Flask that allows students to input their subject grades and credits, compute their Cumulative Grade Point Average (CGPA), and predict their academic performance based on additional input features using a pre-trained machine learning model.



**Project Description:**

---




This project builds a Flask-based web application that:

Calculates CGPA based on the user's input of subject grades and credit hours.

Predicts student performance using a trained machine learning model.

Has a user-friendly web interface built with HTML, CSS, and Flask templates.

Ensures error handling for invalid inputs (e.g., mismatched credit totals, missing values).

Runs on a web server (accessible via 0.0.0.0:8080).

This application can be further enhanced by deploying it on Google Cloud, AWS, or a dedicated server with a domain name for public access.

In [ ]:
!pip install pyngrok
!ngrok authtoken 2rwq2V6fDyu8gFhbDZDm4567rx5mxxwhBwo_4gh6fghwPq1yP8imGyUA4Z1xPN455555


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, render_template, request
from pyngrok import ngrok
import os

app = Flask(__name__)

# Function to get grade points
def get_grade_points(grade):
    grade_mapping = {
        'O': 10,  # 91-100
        'A+': 9,  # 81-90
        'A': 8,  # 71-80
        'B+': 7,  # 61-70
        'B': 6,  # 51-60
        'C': 5   # 40-50
    }
    return grade_mapping.get(grade, 0)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        try:
            num_subjects = int(request.form['num_subjects'])
            total_credits = int(request.form['total_credits'])
            total_weighted_score = 0
            total_subject_credits = 0

            for i in range(num_subjects):
                subject_credits = int(request.form[f'credit_{i}'])
                grade = request.form[f'grade_{i}']

                grade_points = get_grade_points(grade)
                total_weighted_score += grade_points * subject_credits
                total_subject_credits += subject_credits

            if total_subject_credits != total_credits:
                return render_template('index.html', error="Invalid count of credits. Total does not match.")

            if total_subject_credits == 0:
                return render_template('index.html', error="Total subject credits cannot be zero.")

            cgpa = total_weighted_score / total_subject_credits
            return render_template('index.html', cgpa=f"Your CGPA is: {cgpa:.2f}")
        except ValueError:
            return render_template('index.html', error="Please enter valid numbers for credits and subjects.")

    return render_template('index.html')

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"Public URL: {public_url}")
    app.run()

# HTML with embedded styles
html_code = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CGPA Calculator</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            text-align: center;
            margin: 0;
            padding: 20px;
        }
        .container {
            background: white;
            width: 50%;
            margin: auto;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        input, select {
            width: 80%;
            padding: 10px;
            margin: 5px 0 15px 0;
            border-radius: 5px;
            border: 1px solid #ddd;
        }
        .subject-container {
            display: flex;
            flex-direction: column;
            align-items: center;
            margin: 20px 0;
            padding: 15px;
            background: #f9f9f9;
            border-radius: 5px;
            box-shadow: 0px 0px 5px rgba(0, 0, 0, 0.1);
            width: 90%;
            margin-left: auto;
            margin-right: auto;
        }
        button {
            background-color: #28a745;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            margin-top: 10px;
        }
        .result {
            margin-top: 15px;
            font-size: 20px;
            font-weight: bold;
        }
    </style>
    <script>
        function generateInputs() {
            let numSubjects = document.getElementById("num_subjects").value;
            let totalCredits = document.getElementById("total_credits").value;
            let container = document.getElementById("subjects-container");
            container.innerHTML = "";
            document.getElementById("initial-inputs").style.display = "none";

            for (let i = 0; i < numSubjects; i++) {
                container.innerHTML += `
                    <div class="subject-container">
                        <label>Credits for Subject ${i+1}:</label>
                        <input type="number" name="credit_${i}" required>
                        <label>Grade:</label>
                        <select name="grade_${i}" required>
                            <option value="O">O</option>
                            <option value="A+">A+</option>
                            <option value="A">A</option>
                            <option value="B+">B+</option>
                            <option value="B">B</option>
                            <option value="C">C</option>
                        </select>
                    </div>
                `;
            }
            container.innerHTML += '<button type="submit">Calculate CGPA</button>';
        }
    </script>
</head>
<body>
    <div class="container">
        <h2>CGPA Calculator</h2>
        <form method="POST">
            <div id="initial-inputs">
                <label>Total Credits:</label>
                <input type="number" id="total_credits" name="total_credits" required>
                <label>Number of Subjects:</label>
                <input type="number" id="num_subjects" name="num_subjects" oninput="generateInputs()" required>
            </div>
            <div id="subjects-container"></div>
        </form>
        {% if cgpa %}
            <p class="result">{{ cgpa }}</p>
        {% elif error %}
            <p class="result" style="color: red;">{{ error }}</p>
        {% endif %}
    </div>
</body>
</html>
'''

# Save the HTML file
with open("templates/index.html", "w") as f:
    f.write(html_code)
